<a href="https://colab.research.google.com/github/fshiau/comp-geno-proj/blob/main/src/2_hmm%20/2.0_fit-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  data = "/content/drive/MyDrive/ComputationalGenomics/"
except:
  data = "/Users/fionshiau/Library/CloudStorage/GoogleDrive-fioncshiau@gmail.com/My Drive/ComputationalGenomics"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install pomegranate;
!pip install watermark;

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('whitegrid')
import numpy as np
import pandas as pd
import os
import glob

from pomegranate import *

np.random.seed(0)
np.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,scipy,pomegranate

ModuleNotFoundError: ignored

In [ ]:
assays = os.listdir(data)

In [ ]:
obs = [pd.read_table(glob.glob(os.path.join(data,i,'*binary'))[0],header = None, 
                     names= ['chr','start','end',i]) for i in assays if i!= 'README']

In [ ]:
for i in range(len(obs)):
  obs[i].index = obs[i].loc[:,'chr'].values + '_' + obs[i].loc[:,'start'].values.astype(str) + '-' + obs[i].loc[:,'end'].values.astype(str)
  

In [ ]:
obs = pd.concat(obs,axis=1)

In [ ]:
obs.columns

Index(['chr', 'start', 'end', 'H3K4me2', 'chr', 'start', 'end', 'H3K9Ac',
       'chr', 'start', 'end', 'H3K4me3', 'chr', 'start', 'end', 'ATAC-seq',
       'chr', 'start', 'end', 'H3K4me1', 'chr', 'start', 'end', 'H3K27Ac',
       'chr', 'start', 'end', 'HiC', 'chr', 'start', 'end', 'H3K27me3', 'chr',
       'start', 'end', 'RNA-Pol_ChIPseq'],
      dtype='object')

In [ ]:
obs = obs.loc[:,~obs.columns.duplicated()].copy()


In [ ]:
obs

,chr,start,end,H3K4me2,H3K9Ac,H3K4me3,ATAC-seq,H3K4me1,H3K27Ac,HiC,H3K27me3,RNA-Pol_ChIPseq
chr1_0-200,chr1,0,200,0,0,0,0,0,0,0,0,0
chr1_200-400,chr1,200,400,0,0,0,0,0,0,0,0,0
chr1_400-600,chr1,400,600,0,0,0,0,0,0,0,0,0
chr1_600-800,chr1,600,800,0,0,0,0,0,0,0,0,0
chr1_800-1000,chr1,800,1000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
chr22_51303600-51303800,chr22,51303600,51303800,0,0,0,0,0,0,0,0,0
chr22_51303800-51304000,chr22,51303800,51304000,0,0,0,0,0,0,0,0,0
chr22_51304000-51304200,chr22,51304000,51304200,0,0,0,0,0,0,0,0,0
chr22_51304200-51304400,chr22,51304200,51304400,0,0,0,0,0,0,0,0,0


In [ ]:
os.path

<module 'posixpath' from '/Users/fionshiau/opt/miniconda3/envs/hmm/lib/python3.10/posixpath.py'>

In [ ]:
obs.iloc[0:5,:]

,chr,start,end,H3K4me2,H3K9Ac,H3K4me3,ATAC-seq,H3K4me1,H3K27Ac,HiC,H3K27me3,RNA-Pol_ChIPseq
chr1_0-200,chr1,0,200,0,0,0,0,0,0,0,0,0
chr1_200-400,chr1,200,400,0,0,0,0,0,0,0,0,0
chr1_400-600,chr1,400,600,0,0,0,0,0,0,0,0,0
chr1_600-800,chr1,600,800,0,0,0,0,0,0,0,0,0
chr1_800-1000,chr1,800,1000,0,0,0,0,0,0,0,0,0


In [ ]:
os.makedirs(os.path.join("/Users/fionshiau/comp_geno",'merged'),exist_ok=True)

In [ ]:
local_path = "/Users/fionshiau/comp_geno"
obs.to_csv(os.path.join(local_path,'merged','merged_1.csv'))

In [ ]:
chr1 = obs.loc[obs.chr=='chr1',:]

In [ ]:
model = HiddenMarkovModel.from_samples(IndependentComponentsDistribution, n_components=20, X=chr1.iloc[:, 3:].to_numpy())

/var/folders/zb/k8zh8whx36g93snxwz8xsqq00000gn/T/ipykernel_60390/3465380637.py:1: RuntimeWarning: invalid value encountered in divide
  model = HiddenMarkovModel.from_samples(IndependentComponentsDistribution, n_components=20, X=chr1.iloc[:, 3:].to_numpy())


ValueError: ignored

In [ ]:
dist = MultivariateGaussianDistribution.from_samples(obs.iloc[:, 3:].to_numpy())


In [ ]:
dist

{
    "class" : "Distribution",
    "name" : "MultivariateGaussianDistribution",
    "parameters" : [
        [
            0.12134942032649648,
            0.12134935090701413,
            0.12137024617120316,
            0.12140370636169855,
            0.12137788231426228,
            0.12140453939548682,
            0.12139523718485117,
            0.12139607021863943,
            0.12137024617120316
        ],
        [
            [
                0.10662373851291976,
                0.10128705481136609,
                0.09666722115325244,
                0.09392595058925125,
                0.09231174922251781,
                0.09116767462813201,
                0.090252327439942,
                0.08947084065838028,
                0.08879137262156474
            ],
            [
                0.10128705481136609,
                0.10662368594146047,
                0.10132902549681877,
                0.09670398786193933,
                0.0939068785140741,
              

In [ ]:
#https://github.com/keoughkath/LAD_atlas/blob/main/HMM_QC.ipynb as a reference to use bedgraph files in pomegranate 

states_dict = {
        2:['nonLAD','LAD'],
        3:['nonLAD','T2-LAD','T1-LAD'],
        4:['nonLAD','T3-LAD','T2-LAD','T1-LAD'],
        5:['nonLAD','T4-LAD','T3-LAD','T2-LAD','T1-LAD']
    }

['nonLAD', 'LAD']